# Analysis Parliamentary Documents

## A.K.A My misadventure at playing with gensim

# Who am I

* My name is sweemeng
* I am a software developer. 
* I used python mostly and take up some sysadmin role
* I work with a startup 
* I also volunteered for a small NGO called Sinar Project


## Warning I am more hacker than data scientist. So here goes

# Let's talk about LDA 

* It is essentially a statistical model of documents.
* It works with a bag of words models.
* It is a pretty generic model as it don't use linguistic or parsing. 
* that's the idea anyway
* That what's gensim provide

# About the documents

* it is a set of document from Parliament. 
* It is called parliamentary reply, where Representative ask question to government
* It is cool because:
  * It is in Malay. It is not explored as much than english
  * The language in the doc is formal, so it is cleaner(in theory)

# Here's where you can get the documents. 

* The source of documents is from a shared directory from Dr Ong Kian Ming an MP
* We need to download it manually then process it with the tools:
  * https://github.com/leowmjw/parliamentMY-QA-blast
* The pdf is hosted at https://pardocs.sinarproject.org/https://github.com/leowmjw/parliamentMY-QA-blast

# The document's is pdf so let's preprocess

Let's load some dependency

In [1]:
import os

In [2]:
import PyPDF2

# Now we process pdf

Let's read one file

In [3]:
docs = PyPDF2.PdfFileReader(open("docs/results/20170307__DR_JawabLisan/soalan-14.pdf", "rb"))

In [4]:
docs.numPages 

2

In [5]:
page_1 = docs.getPage(0)

In [6]:
page_1.extractText()

'PERTANYAAN DARIPADA TARIKH RUJUKAN SO ALAN PEMBERITAHU PERTANYAAN DEWAN RAKYAT, MALAYSIA LISAN SOALAN NO: 14 PUAN TERESA KOK SUH SIM [SEPUTEH] SELASA, 7 MAC 2017 L 1 [PR-1351-L00118] Puan Teresa Kok Suh Sim [Seputeh] minta MENTERI WILAYAH PERSEKUTUAN menyatakan prosedur penjualan tanah D BKL. Kenapakah tender terbuka tidak dilaksanakan daripada segi penjualan tanah Kerajaan. '

Processing all the file could take time. and complicated. You are not here for this!

# Actually there is processing stage 2. 

A few thing you need to know

* LDA is a stasticial model, it works with number. A matrices 
* You need to tokenize into list and calculated
* some processing within is needed, like using stopwords etc. 

# Now let's actually process it

First some setup

In [7]:
import data_extractor # This is a script to process pdf

In [8]:
docs = []

In [9]:
index = [] # I show you later

In [10]:
stopwords = data_extractor.stop_words()

In [11]:
stopwords

['ada',
 'adanya',
 'adalah',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'diantaranya',
 'antara',
 'antaranya',
 'diantara',
 'apa',
 'apaan',
 'mengapa',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'atau',
 'ataukah',
 'ataupun',
 'bagai',
 'bagaikan',
 'sebagai',
 'sebagainya',
 'bagaimana',
 'bagaimanapun',
 'sebagaimana',
 'bagaimanakah',
 'bagi',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'sebaliknya',
 'banyak',
 'sebanyak',
 'beberapa',
 'seberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'sebegini',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'sebegitu',
 'belum',
 'belumlah',
 'sebelum',
 'sebelumnya',
 'sebenarnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'betulkah',
 'sebetulnya',
 'biasa',
 'biasanya',
 'bila',
 'bilakah',
 'bisa',
 'bisakah',
 'sebisanya',
 'boleh',
 'bolehkah',
 'bolehlah',
 'buat',
 'bukan',
 'bukankah',
 'bukanlah',


Let's try to clean up by adding more word to stopwords

In [12]:
stopwords.extend(['menteri','pertua', 'soalan', 'berhormat', 'pihak', 'dato', 'yb', 'jawapan','lisan', 'pertanyaan'
                 ,'tuan', 'kerajaan'])

# Now tokenize and filter

LDA doesn't work on sentences. It work on a set of word. So we need to tokenize it

In [13]:
from nltk.tokenize import RegexpTokenizer

In [14]:
tokenizer = RegexpTokenizer(r'(?:RM)*\d+(?:,|.\d+)*|\w+') # try guess what this do

In [15]:
for doc_path in data_extractor.document_walker(path="docs/results/"): # Get all pdf file in this path
    doc = data_extractor.extract_text_single(doc_path) #turn it into text
    # First process
    text = tokenizer.tokenize(doc.lower())
    #text = tokenizer.tokenize(doc)
    index.append((doc_path, doc, text))
    text = [ word for word in text if word not in stopwords ]
    docs.append(text)

In [16]:
counter = {}

In [17]:
for sent in docs:
    for word in sent:
        if word not in counter:
            counter[word] = 1
        else:
            counter[word] = counter[word] + 1

In [18]:
clean_docs = []

In [19]:
for sent in docs:
    temp = []
    for word in sent:
        if counter[word] > 5:
            temp.append(word)
    clean_docs.append(temp)
    

In [20]:
docs = clean_docs

# Now create corpus

* We will be using gensim corpora tools
* Here we turn into to a bag of words format instead of sentense

In [21]:
from gensim import corpora

In [22]:
dictionary = corpora.Dictionary(docs)

In [23]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

# We can actually save results

In [24]:
corpora.MmCorpus.serialize("malaysian_parl_clean.mm", corpus)

In [25]:
dictionary.save("malaysian_parl_clean.dict")

In [26]:
import json 

# I just want to store some original data to show things
json.dump(index, open("index.json", "w"))

# Let's create a LDA model

In [27]:
from gensim import models

In [28]:
corpus = corpora.MmCorpus('malaysian_parl_clean.mm')

In [29]:
model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=60)

And that's it!

# Let's visualize

In [30]:
import pyLDAvis

In [31]:
import pyLDAvis.gensim

In [32]:
parl_data = pyLDAvis.gensim.prepare(model, corpus, dictionary)

In [33]:
pyLDAvis.display(parl_data)

# Cool pics, what's the point

Well Lda model can do a few more things. First let's do some proocessing for the following text

In [34]:
index[20][1]

'SOALAN NO: 13 PEMBERITAHUAN PERTANYAAN DEWAN RAKYAT JAWAPAN OLEH YB DATUK SERI DR. S. SUBRAMANIAM MENTERI KESIHATAN MALAYSIA PERTANYAAN DARIPADA TARIKH SO ALAN LISAN YB DATUK HAJI SHABUDIN BIN YAHAYA .[ TASEK GELUGOR] 21MAC2016 YB Datuk Haji Shabudin Bin Yahaya [ Tasek Gelugor ] minta MENTERI KESIHAT AN menyatakan apakah langkah proaktif Kementerian bagi mengawal peningkatan permasalahan mental dalam kalangan masyarakat Malaysia terutamanya golongan yang berumur diantara 16 hingga 19 tahun dan yang berpendapatan rendah Tuan Yang Di Pertua, Mengikut Kajian National Health and Morbidity SuNey (NHMS) yang dijalankan oleh Kementerian Kesihatan Malaysia pada 2015 prevalen masalah kesihatan mental dalam kalangan orang dewasa berumur 16 keatas adalah 29.2%. Waiau bagaimana pun peratus ini merujuk kepada mereka yang mengalami masalah kesihatan mental dan BUKAN penyakit mental 1 l-3  Tuan Yang Di Pertua, Kementerian Kesihatan Malaysia (KKM) secara berterusan \nmemberi perkhidmatan saringan, pe

# Processing again

In [35]:
bow = model.id2word.doc2bow(index[20][2])

In [36]:
doc_topics, word_topics, phi_value = model.get_document_topics(bow, per_word_topics=True)

In [37]:
doc_topics

[(3, 0.25569909269332464),
 (39, 0.34829747380320042),
 (48, 0.39322565572569745)]

In [38]:
model.show_topic(16)

[('orang', 0.018023416905676479),
 ('tahun', 0.012566754521177354),
 ('negara', 0.01083859534574287),
 ('malaysia', 0.0099519719152985266),
 ('jumlah', 0.0081634239026279198),
 ('rakyat', 0.0077977531453666303),
 ('2017', 0.0076582774969914998),
 ('bin', 0.0057103339792941429),
 ('2', 0.0054898327270906421),
 ('secara', 0.0054345456834582664)]

You can group documents with this, add some labels for people to use. 

# You can also use this for similarity search

In [39]:
from gensim import similarities

In [40]:
lsi_model = models.LsiModel(corpus,id2word=dictionary, num_topics=60)

In [41]:
lsi_vec = lsi_model[bow]

In [42]:
sim_index = similarities.MatrixSimilarity(lsi_model[corpus])

In [43]:
sorted(enumerate(sim_index[lsi_vec]), key=lambda item: -item[1])

[(20, 1.0),
 (449, 0.85295826),
 (70, 0.82153881),
 (620, 0.81313264),
 (53, 0.79868686),
 (66, 0.7823832),
 (630, 0.77441049),
 (145, 0.75901884),
 (648, 0.74386263),
 (141, 0.70755166),
 (244, 0.67792171),
 (639, 0.65784776),
 (253, 0.64702773),
 (122, 0.62400365),
 (659, 0.61703748),
 (486, 0.61342448),
 (556, 0.60846949),
 (24, 0.59862334),
 (401, 0.59284282),
 (634, 0.57304227),
 (46, 0.56197268),
 (521, 0.55622864),
 (561, 0.55371797),
 (519, 0.5531162),
 (52, 0.5310961),
 (560, 0.52782822),
 (71, 0.51472551),
 (111, 0.5046013),
 (35, 0.49564499),
 (499, 0.47556192),
 (655, 0.46874407),
 (211, 0.46855766),
 (190, 0.45854536),
 (484, 0.45181519),
 (110, 0.45032179),
 (208, 0.44070983),
 (421, 0.42858005),
 (231, 0.42469656),
 (138, 0.42346501),
 (198, 0.41502425),
 (118, 0.41324839),
 (164, 0.40190083),
 (646, 0.39648896),
 (294, 0.39480734),
 (285, 0.38955802),
 (4, 0.3877745),
 (492, 0.38582602),
 (527, 0.38253501),
 (410, 0.38209933),
 (584, 0.38119569),
 (123, 0.37908977),
 (2

# Let's look at the docs to see how well it works

In [44]:
index[20][1]

'SOALAN NO: 13 PEMBERITAHUAN PERTANYAAN DEWAN RAKYAT JAWAPAN OLEH YB DATUK SERI DR. S. SUBRAMANIAM MENTERI KESIHATAN MALAYSIA PERTANYAAN DARIPADA TARIKH SO ALAN LISAN YB DATUK HAJI SHABUDIN BIN YAHAYA .[ TASEK GELUGOR] 21MAC2016 YB Datuk Haji Shabudin Bin Yahaya [ Tasek Gelugor ] minta MENTERI KESIHAT AN menyatakan apakah langkah proaktif Kementerian bagi mengawal peningkatan permasalahan mental dalam kalangan masyarakat Malaysia terutamanya golongan yang berumur diantara 16 hingga 19 tahun dan yang berpendapatan rendah Tuan Yang Di Pertua, Mengikut Kajian National Health and Morbidity SuNey (NHMS) yang dijalankan oleh Kementerian Kesihatan Malaysia pada 2015 prevalen masalah kesihatan mental dalam kalangan orang dewasa berumur 16 keatas adalah 29.2%. Waiau bagaimana pun peratus ini merujuk kepada mereka yang mengalami masalah kesihatan mental dan BUKAN penyakit mental 1 l-3  Tuan Yang Di Pertua, Kementerian Kesihatan Malaysia (KKM) secara berterusan \nmemberi perkhidmatan saringan, pe

In [45]:
index[449][1]

"SOALAN NO: 13 PEMBERITAHUAN PERTANYAAN DEWAN RAKYAT JAWAPAN OLEH YB DATUK SERI DR. S. SUBRAMANIAM MENTERI KESIHATAN MALAYSIA PERTANYAAN DARIPADA TARIKH SO ALAN LISAN YB DATO' SRI ABDUL AZEEZ BIN ABDUL RAHIM [ BALING ] 16 MAC 2017 YB Dato' Sri Abdul Azeez Bin Abdul Rahim [ Baling ] minta MENTERI KESIHATAN menyatakan perancangan jangka pendek dan jangka panjang kementerian dalam menaik taraf fasiliti-fasiliti kesihatan sedia ada yang umumnya kita ketahui sebahagian besar klinik-klinik kesihatan, klinik-klinik desa ini telah dibina pada awal tahun 70an lagi dan masih beroperasi sehingga ke hari ini. Tuan Yang di-Pertua, Untuk makluman, sehingga Disember 2015 Kementerian Kesihatan Malaysia (KKM) mempunyai 143 hospital dan institusi perubatan , 1,061 Klinik Kesihatan (termasuk klinik ibu dan anak) dan 1,808 Klinik Desa yang berlainan usia. Usaha-usaha menaiktaraf dan menggantikan fasiliti kesihatan tersebut dijalankan oleh KKM secara berterusan mengikut keperluan pada setiap Rancangan Mala

Not bad... :Obama face:

# So the conclusion?

* Text analysis tools is easier to use
* But getting it right is still hard
* As you can see, the mileage varies. 
* Still a tool that can automatically apply labels, and compare similarities is still useful. 
* I really need to stop procrastinate on my project use RNN